In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.transform import pyramid_gaussian

import sscPhantom 
# import xraydb

In [ ]:
def calculate_delta_beta_xraydb(material,energy):
    
    if material == 'Ni': # nickel
        density = 8.908 # g/cm^3
    elif material == 'Si': # silicon
        density = 2.329002
    elif material == 'W': # tungsten
        density = 19.25
    elif material == 'Au': # gold
        density = 19.3
    elif material == 'C': # carbon
        density = 2.2
        
    delta, beta, attenuation_length = xraydb.xray_delta_beta(material,density,energy)
    return delta, beta 

def get_phantom(inputs):

    params = { 'HowMany': 10,
               'radius': 0.07,
               'Rtorus': 0.5,
               'rtorus': 0.1}
    
    phantom2, phantom1 = sscPhantom.donuts.createDonuts( inputs["matrix_size"],inputs["n_cpus"], inputs["energy"], params )
    
    print(np.max(phantom1), np.mean(phantom1), np.min(phantom1))
    print(np.max(phantom2), np.mean(phantom2), np.min(phantom2))    
    
    # phantom2 = phantom2/np.max(phantom2)
    # delta, beta = calculate_delta_beta_xraydb('C',inputs["energy"]*1e3)
    # print(delta,beta)
    # phantom2 = beta*phantom2*np.exp(1j*delta*phantom2)

    # mean = np.mean(phantom1[phantom1>0])
    # print('mean',mean)
    
    # Gold
    # phantom1 = phantom1/np.max(phantom1)

    donut = phantom1 + phantom2
    phantom = np.swapaxes(donut,1,0)

    return phantom

In [ ]:
inputs = {}
inputs["matrix_size"] = 256
inputs["n_cpus"] = 32 
inputs["energy"] = 9

In [ ]:
phantom = get_phantom(inputs)
phantom.shape

In [ ]:
proj = np.sum(phantom,axis=0)

fig, ax = plt.subplots(dpi=150)
ax.imshow(np.abs(proj))

In [6]:
from scipy.ndimage import rotate

def rotate_matrix(matrix,angle,rotation_point=None):
    
    if rotation_point == None:
        rotation_point = (matrix.shape[0]//2,matrix.shape[1]//2)

    real_part, imag_part = matrix.real, matrix.imag 
    rotated_matrix = rotate(real_part,angle,axes=(0,2),reshape=False)
    
    return rotated_matrix

angles = np.linspace(0,180,360+1)
# angles = [0,1,2,3,4]
sinogram = np.empty((len(angles),phantom.shape[0],phantom.shape[1]))
for i, angle in enumerate(angles):
    phantom_rotated = rotate_matrix(phantom,angle)
    proj = np.sum(phantom_rotated,axis=0)
    sinogram[i] = proj
    print(i,angle)
    # fig, ax = plt.subplots(dpi=100)
    # ax.imshow(proj)
    # plt.show()
    # plt.close()
    
np.save('/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/regular_angles_donut.npy',sinogram)    


349 174.5
350 175.0
351 175.5
352 176.0
353 176.5
354 177.0
355 177.5
356 178.0
357 178.5
358 179.0
359 179.5
360 180.0


In [20]:
print(sinogram.shape)
angles = np.linspace(0,180,360+1)
a = np.linspace(0,360,360+1)
angles = np.vstack((a,angles)).T
print(angles.shape)
np.save('/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/angles.npy',angles)

(361, 256, 256)
(361, 2)


In [32]:
shifted_sinogram = np.empty_like(sinogram)
pixel_shifts = []
for i, proj in enumerate(sinogram):

    factor = 25
    
    px = int((np.random.rand(1)-0.5)*factor)
    py = int((np.random.rand(1)-0.5)*factor)
    
    pixel_shifts.append((py,px))

    shifted = np.roll(proj, shift=px, axis=1)
    shifted_sinogram[i] = np.roll(shifted, shift=py, axis=0)
     
    # fig, ax = plt.subplots(1,2,dpi=100)
    # ax[0].imshow(proj)
    # ax[1].imshow(shifted)

np.save('/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/regular_angles_donut_shifted.npy',shifted_sinogram)    
np.save('/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/pixel_shifts.npy',np.asarray(pixel_shifts))

In [24]:
%matplotlib widget
import sscCdi

In [92]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
o_path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/phantom_donut.hdf5"

path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/regular_angles_donut_shifted.npy"
shifted_data = np.load(path)

path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/angles.npy"
angles = np.load(path)

path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/regular_angles_donut.npy"
data = np.load(path)

path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_donut/pixel_shifts.npy"
shifts = np.load(path)

def make_complex(data):
    data = data/np.max(data)
    data[data<1e-2] = 0
    data += 1e-2
    # data += 0.1*np.mean(data)*np.random.rand(*data.shape)
    data = data*np.exp(-1j*0.7*np.pi*(data))
    # data[np.abs(np.angle(data))<1e-3] = 1e-2
    return data

data = make_complex(data)
shifted_data = make_complex(shifted_data)


angles = np.hstack((angles,angles))
angles[:,2] = angles[:,3]
angles[:,3] = angles[:,3]*np.pi/180
angles[:,1] = 0
angles.shape, angles


with h5py.File(o_path, 'w') as f:
    dset = f.create_dataset('recon/object', data=shifted_data)
    dset = f.create_dataset('recon/angles', data=angles)
    dset = f.create_dataset('recon/original_object', data=data)
    dset = f.create_dataset('recon/pixel_shifts', data=shifts)
    
    f.close()

In [93]:
sscCdi.deploy_visualizer(shifted_data,type='phase',title="Cropped sinogram",cmap='gray')